In [240]:
from collections import defaultdict, deque
import random
import math

import numpy as np

class City:
    id = 0
    def __init__(self):
        self.X = random.randint(-100, 100)
        self.Y = random.randint(-100, 100)
        self.Z = random.randint(0, 50)
        self.coordinates = np.array((self.X, self.Y, self.Z))
        self.id = City.id
        City.id += 1
    
class Graph:
    
    def __init__(self, cities):
        self.all_paths = []
        self.adjList = defaultdict(list)
        self.sym_matrix = self.generate_sym_matrix(cities)
        self.matrix_80 = self.generate_80_matrix(cities)
        self.asym100_matrix = self.generate_asym_matrix(cities)
        self.asym80_matrix = self.generate_80asym_matrix(cities)

    def generate_sym_matrix(self, cities):
        sym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i != j:
                    row.append(1)
                else:
                    row.append(0)
            sym_matrix.append(row)
        return np.array(sym_matrix)
    
    def generate_80_matrix(self, cities):
        matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i != j:
                    row.append(1)
                else:
                    row.append(0)
            matrix.append(row)
        matrix = np.array(matrix)
        total_elements = matrix.size
        num_to_change = int(0.2 * total_elements)
    
        indices_to_change = np.random.choice(total_elements, num_to_change, replace=False)
        matrix_flattened = matrix.flatten()
    
        for index in indices_to_change:
            if matrix_flattened[index] == 1:
                matrix_flattened[index] = 0
    
        return matrix_flattened.reshape(matrix.shape)
    
    def generate_asym_matrix(self, cities):
        asym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i == j:
                    row.append(0)
                elif cities[i].Z < cities[j].Z and i != j:
                    row.append(2)
                elif cities[i].Z >= cities[j].Z and i != j:
                    row.append(1)
            asym_matrix.append(row)
        return np.array(asym_matrix)
    

    def generate_80asym_matrix(self, cities):
        asym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i == j:
                    row.append(0)
                elif cities[i].Z < cities[j].Z and i != j:
                    row.append(2)
                elif cities[i].Z >= cities[j].Z and i != j:
                    row.append(1)
            asym_matrix.append(row)
        asym_matrix = np.array(asym_matrix)
        total_elements = asym_matrix.size
        num_to_change = int(0.2 * total_elements)
    
        indices_to_change = np.random.choice(total_elements, num_to_change, replace=False)
        matrix_flattened = asym_matrix.flatten()
    
        for index in indices_to_change:
            if matrix_flattened[index] == 1 or matrix_flattened[index] == 2:
                matrix_flattened[index] = 0
    
        return matrix_flattened.reshape(asym_matrix.shape)

    def calculate_distance(self, city1, city2):
        return np.linalg.norm(city1.coordinates - city2.coordinates)
    
    def calculate_distance_asym(self, city1, city2):
        if city1.Z < city2.Z:
            #print("Going up")
            return 1.1 * math.sqrt((city2.X - city1.X) ** 2 + (city2.Y - city1.Y) ** 2 + (city2.Z - city1.Z) ** 2)
        elif city1.Z > city2.Z:
            #print("Going down")
            return 0.9 * math.sqrt((city2.X - city1.X) ** 2 + (city2.Y - city1.Y) ** 2 + (city2.Z - city1.Z) ** 2)
        else:
            #print("Same height")
            return np.linalg.norm(city1.coordinates - city2.coordinates)



    def addEdge(self, u, v):
            self.adjList[u].append(v)


    def createEdge(self, matrix):
        for i in range(0, len(matrix)):
            for j in range(0, len(matrix)):
                if matrix[i][j] == 1 or matrix[i][j] == 2:
                    self.addEdge(i, j)
                    #print(f"Created edge between {i} and {j}")

                    

    def bfs(self, startNode):
        queue = deque([(startNode, [startNode])])
        paths = []

        while queue:
            currentNode, path = queue.popleft()
            #print(f"BFS: {path}")
            
            if len(path) == len(self.adjList) + 1:
                paths.append(path)
                print(path)
                continue

            for neighbour in self.adjList[currentNode]:
                if neighbour not in path:
                    queue.append((neighbour, path + [neighbour]))
                elif len(path) == len(self.adjList) and neighbour == 0:
                    queue.append((neighbour, path + [neighbour]))
        return paths
    

    def dfs(self, startNode):
        queue = deque([(startNode, [startNode])])
        paths = []

        while queue:
            currentNode, path = queue.pop()
            #print(f"DFS: {path}")
            
            if len(path) == len(self.adjList) + 1:
                paths.append(path)
                print(path)
                continue

            for neighbour in self.adjList[currentNode]:
                if neighbour not in path:
                    queue.append((neighbour, path + [neighbour]))
                elif len(path) == len(self.adjList) and neighbour == 0:
                    queue.append((neighbour, path + [neighbour]))
        return paths


    # def dfs_help(self, currentNode, visited, path, all_paths):
    #     visited.add(currentNode)
    #     path.append(currentNode)

    #     if len(path) == len(self.adjList):
    #         for neighbour in self.adjList[currentNode]:
    #             if neighbour == 0:
    #                 path.append(neighbour)
    #                 all_paths.append(path[:])
    #                 print(path)
    #                 path.pop()
            
    #     for neighbour in self.adjList[currentNode]:
    #         if neighbour not in visited:
    #             self.dfs_help(neighbour, visited, path, all_paths)
            

    #     visited.remove(currentNode)
    #     path.pop()

    # def dfs(self, startNode):
    #     visited = set()
    #     all_paths = []

    #     self.dfs_help(startNode, visited, [], all_paths)
    #     return all_paths

    
    
    def find_shortest_sym_path(self, cities, method):
        min_distance = float('inf')
        path = float('inf')
        distance = 0
        paths = []
        if method == 'bfs':
            print('bfs')
            paths = self.bfs(0)
        elif method == 'dfs':
            print('dfs')
            paths = self.dfs(0)
        #print(f"Paths: {paths}\n\n")
        for i in range(0, len(paths)):
            #print(f"{i}: {paths[i]}")
            for j in range(0, len(paths[i]) - 1):
                #print(f"({paths[i][j]}{paths[i][j + 1]})")
                distance += self.calculate_distance(cities[paths[i][j]], cities[paths[i][j+1]])
            #print(f"Distance in path {paths[i]}: {distance}")
            if distance < min_distance:
                min_distance = distance
                path = i + 1
            distance = 0
        return f'Min distance is: {min_distance} for path {path}'
    


    def find_shortest_asym_path(self, cities, method):
        min_distance = float('inf')
        path = float('inf')
        distance = 0
        paths = []
        if method == 'bfs':
            print('bfs')
            paths = self.bfs(0)
        elif method == 'dfs':
            print('dfs')
            paths = self.dfs(0)
        #print(f"Paths: {paths}\n\n")
        for i in range(0, len(paths)):
            #print(f"{i}: {paths[i]}")
            for j in range(0, len(paths[i]) - 1):
                #print(f"({paths[i][j]}{paths[i][j + 1]})")
                distance += self.calculate_distance_asym(cities[paths[i][j]], cities[paths[i][j+1]])
            #print(f"Distance in path {paths[i]}: {distance}")
            if distance < min_distance:
                min_distance = distance
                path = i + 1
            distance = 0
        return f'Min distance is: {min_distance} for path {path}'
                


In [241]:
cities = []
for i in range(0, 6):
    cities.append(City())
    print(cities[i].Z)

graph = Graph(cities)

# print(graph.sym_matrix)
# print(graph.matrix_80)
# print(graph.asym100_matrix)
print(graph.asym80_matrix)
graph.createEdge(graph.asym80_matrix)


min_distance_bfs = graph.find_shortest_asym_path(cities, 'bfs')
print(min_distance_bfs)

min_distance_dfs = graph.find_shortest_asym_path(cities, 'dfs') 
print(min_distance_dfs)



7
3
25
21
24
16
[[0 1 2 2 0]
 [2 0 0 2 2]
 [0 1 0 1 1]
 [0 1 2 0 2]
 [1 1 2 1 0]]
bfs
[0, 1, 3, 2, 4, 0]
[0, 2, 1, 3, 4, 0]
[0, 2, 3, 1, 4, 0]
[0, 2, 3, 4, 1, 0]
[0, 2, 4, 3, 1, 0]
[0, 3, 2, 1, 4, 0]
[0, 3, 2, 4, 1, 0]
[0, 3, 4, 2, 1, 0]
Min distance is: 383.37588159752374 for path 7
dfs
[0, 3, 4, 2, 1, 0]
[0, 3, 2, 4, 1, 0]
[0, 3, 2, 1, 4, 0]
[0, 2, 4, 3, 1, 0]
[0, 2, 3, 4, 1, 0]
[0, 2, 3, 1, 4, 0]
[0, 2, 1, 3, 4, 0]
[0, 1, 3, 2, 4, 0]
Min distance is: 383.37588159752374 for path 2
